In [ ]:
################################################
# New style 
################################################
import sys
import os

workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")

from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.Plotting import utils as plu
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( plu )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
#importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(RgF)


In [ ]:
print( sys.path )
############################
# Initialize dict for regridding
############################
RgObs={}
yamldir='../YAML/'

In [ ]:
### Make a list of files
import yaml
#from box import Box


with open(f'{yamldir}configure_plotting.yaml', 'r') as file:
    cfgPlot = yaml.safe_load(file)


######################################################
season = 'jja' #cfgPlot['zonal_mean']['season']
valData  = cfgPlot['zonal_mean']['validation']['data']
valYears = cfgPlot['zonal_mean']['validation']['years']
print( f"Season= {season}" )
print( valData , valYears )
fld='Nudge_U'
fld='UTEND_GWDTOT'
fld='UTGWORO'
fld='UTEND_CORE'
fld='UTEND_PHYSTOT'
#fld='UTGW_MOVMTN'
#fld='UTGWSPEC'


flds		= [ 'UTEND_TOT', 'UTEND_PHYSTOT', 'UTEND_CORE', 'UTEND_VDIFF', 'UTEND_GWDTOT', 'UTEND_DCONV', 'UTEND_SHCONV',
         'UTEND_MACROP', 'UTEND_RAYLEIGH',] 
#'UAP:I', 'UBP:I', 'U:I' ]

flds = [ 'Nudge_U' , 'UTEND_GWDTOT', 'UTGWSPEC', 'UTEND_PHYSTOT','UTEND_CORE','UTGW_MOVMTN','UTGWORO',]
flds = [  'UTGWSPEC','UTGW_MOVMTN','UTGWORO','UTEND_CORE',]


In [ ]:

#"""
#exp_file=f'{yamldir}GW_ndg_ppe.yaml'
#xl=['ndg00','ndg01c','ndg02b',]
#xl=['ndg01c', ] #'ndg02b',]
#"""

#exp_file=f'{yamldir}GWppe3.yaml'
#xl= ['tun00'] 

exp_file=f'{yamldir}Utility.yaml'
#xl= ['cfrntmm_t06' , ] #'control_re', ] #'tun00','tun0m1','tun01'] #'ne240x2_QxQsst']
xl= ['control_re', ] #'tun00','tun0m1','tun01'] #'ne240x2_QxQsst']

with open( exp_file, 'r') as file:
    cfg = yaml.safe_load(file)

Al =[]
for x in xl:
    exp, subd, Src, Hkey, Dst, useri = cfg[x]['name'] , cfg[x]['subdir'] , cfg[x]['SrcGrid'] , cfg[x]['Hkey'] , cfg[x]['DstGrid'] , cfg[x]['user'] 
    ymdPat = cfg[x]['ymdPat']
    print( exp, subd, Src, Hkey, Dst, useri )
    print( ymdPat  )
    
    A = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd , 
                     hsPat='cam.h1a' , ymdPat=ymdPat,verbose=True, open_dataset=True )
    A['Src']=Src
    A['Hkey']=Hkey
    if 'gw_parameters' in cfg[x]:
        A['gw_parameters'] = cfg[x]['gw_parameters']
    Al.append( A )
    print( f'{A.exp} Years {len(A.X.time) /12. } \n')


In [ ]:
########################################
# Make seasonal means
########################################

for A in Al:
    print( f'{A.exp} Years {len(A.X.time) /12. }')

    for fld in flds:  
        print ( fld )
        Fa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld=fld,return_time=True)
        A[f'{fld}_s']=Fa_s
        A['years']=years_a
        A['months']=months_a


In [ ]:
########################################
# Make zonal means
########################################
method='C2'

for A in Al:
    for fld in flds:
        Hkey = A['Hkey']
        Src  = A['Src']
        Fa_s = A[f'{fld}_s']

        if ( np.shape(Fa_s) != () ):
            
            if (Hkey == 'yx' ):
                ########################
                # Option 1 from lat-lon regirrdded data
                ########################
        
                lon_a=A.X.lon.values
                lat_a=A.X.lat.values
                lev_a=A.X.lev.values
                zlev_a=-7.0*np.log( lev_a/1_000. )
                # Zonal Av
                Fa_sz=np.average( Fa_s , axis=2 )
            
            elif (Hkey=='c'):
                ########################
                # Option 2 from unstructured SE data
                ########################
                
                Dst = 'fv1x1'
                DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
                lat_a,lon_a = GrU.latlon( scrip= DstInfo['scrip'], Hkey=DstInfo['Hkey'] )
                lev_a=A.X.lev.values
                zlev_a=-7.0*np.log( lev_a/1_000. )
            
                RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=Src, dst=Dst)
                Fa_s_xy=RgF.Horz(xfld_Src=Fa_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
                
                # Zonal Av
                Fa_sz=np.average( Fa_s_xy , axis=2 )
        
            A[f'{fld}_sz'] = Fa_sz
            A['lonx'] = lon_a
            A['latx'] = lat_a
            A['zlev'] = zlev_a
            A['plev'] = lev_a
            A['DateRange'] = f"{(A.X.time[0].values.item() ).strftime('%Y-%m-%d')}_{(A.X.time[-1].values.item() ).strftime('%Y-%m-%d')}"
        else:
            A[f'{fld}_sz'] = -9999999.


In [ ]:
print( Al[0].keys())

In [ ]:
# skip this
"""
####
# Validation
####
Validation_on_control_mgrid = False

if (Validation_on_control_mgrid == True):
    PSc_s = Av.Seasonal( ds=Dc, season=season , fld='PS')
    mgrid={'ps':PSc_s, 'hyam':hyam_c, 'hybm':hybm_c, 'hyai':hyai_c, 'hybi':hybi_c, 'hgrid':'fv0.9x1.25' }
else:
    mgrid=False


valYears='2014' #'*'
Fval = Val.data(fld=fld,season=season,mgrid=mgrid,zlev=True , Years=valYears ) #, Years='1996' ) #, Years='2000' )
if (Fval['rcode'] == 0):
    Fv_s , zlev_v,lat_v,lon_v, years_Uv = Fval['aa'], Fval['lev'], Fval['lat'], Fval['lon'], Fval['years']
    Fv_sz=np.average( Fv_s , axis=2 )
"""


In [ ]:
import matplotlib.colors as mcolors

# Sample data
x = np.linspace(-2, 2, 100)
y = np.linspace(-2, 2, 100)
X, Y = np.meshgrid(x, y)
Z = np.exp(X**2 - Y**2) * 100  # Some arbitrary function

# Uneven contour levels
levels = [0, 1, 10, 50, 100, 500, 1000]

# Create a colormap normalization that spaces colors evenly
cmap = plt.cm.viridis  # Or any other colormap
norm = mcolors.BoundaryNorm(boundaries=levels, ncolors=cmap.N, clip=False)

# Plot the contour
fig, ax = plt.subplots()
cf = ax.contourf(X, Y, Z, levels=levels, cmap=cmap, norm=norm)
plt.colorbar(cf, ticks=levels)  # Ensure colorbar ticks match levels


In [ ]:
import matplotlib.colors as mcolors



#nxplo,nyplo=4,1
#fig = plt.figure(figsize=(24, 5))

numx=len( Al )
numf=len( flds )
nxplo,nyplo=numx*numf+1,2
fig = plt.figure(figsize=(8*nxplo, 18))
print(fld)
flev,dlev,scale= plu.contour_intervals( 'UTGWORO' , zonal_mean=1 )
print(flev)
cmap='bwr' #'gist_ncar'

# Create a colormap normalization that spaces colors evenly
cmap = plt.cm.bwr  #gist_ncar  # Or any other colormap
norm = mcolors.BoundaryNorm(boundaries=flev, ncolors=cmap.N, clip=False)


titlesize=18

plotTop=65 #85
xlim=[-90,0]
BoxParm =['alpha_gw_movmtn', 'effgw_rdg_resid','use_gw_rdg_resid']
BoxParm =['use_gw_convect_dp','use_gw_movmtn_pbl','use_gw_front','use_gw_rdg_resid','use_gw_rdg_beta','use_gw_oro']
BoxParm =['use_gw_movmtn_pbl','use_gw_rdg_resid','movmtn_source','alpha_gw_movmtn', 'effgw_movmtn_pbl',]  #'effgw_rdg_resid',]
BoxParm =['use_gw_movmtn_pbl','movmtn_source','alpha_gw_movmtn', 'effgw_movmtn_pbl',]  #'effgw_rdg_resid',]

ix=0

for A in Al:
    for fld in flds:
        Fa_s = A[f'{fld}_s']
        if ( np.shape(Fa_s) != () ):

            Fa_sz = A[f'{fld}_sz']  # -Al[0].F_sz
            lon_a = A['lonx']
            lat_a = A['latx']
            zlev_a = A['zlev']
            years_a = A['years']
            if 'gw_parameters' in A:
                parameters=A['gw_parameters']
            else:
                parameters=None
        
            # Create 2D arrays
            Lat_a, Zlev_a = np.meshgrid(lat_a, zlev_a)
            
            # Adding the hatching for the sponge layer
            sponge_layer = np.zeros_like(Fa_sz)
            sponge_layer[(Zlev_a >= 62) & (Zlev_a <= 85)] = 1
            
            title = f"<{A.exp}> {fld} \n {season.upper()} {years_a[0]}-{years_a[-1]}"
            n=ix+1
            Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo, ypan=0.8 ) 
            ax1 = fig.add_axes( Axes1 )
            co1=ax1.contourf(lat_a,zlev_a, scale*Fa_sz ,levels=flev, cmap=cmap , norm=norm, extend='both' )
            co2=ax1.contour(lat_a,zlev_a,  scale*Fa_sz ,levels=flev, colors='black')
            #ax1.set_title(f"Control <{A.exp}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
            ax1.set_title( title ,fontsize=titlesize )
            ax1.set_xlim( xlim )
            ax1.set_ylim(0,plotTop)
            cb=plt.colorbar(co1 , ticks=flev , format='%0.3f')
            ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
                fontsize=20, fontweight='bold', va='top')
            ax1.set_xlabel( 'Latitude', fontsize=20)
            ax1.set_ylabel( 'Z (km)', fontsize=20)
            
            # Sponge layer hatching
            ax1.contourf(lat_a, zlev_a, sponge_layer, levels=[0.5, 1.5], hatches=['//'], alpha=0)
            
            Pu.param_text_box( ax=ax1, parameters=parameters, select=BoxParm , xpo=0.65, ypo=-.1 ) 
            #Pu.param_text_box( ax=ax1, parameters=parameters, select=BoxParm , xpo=0.9, ypo=1.15 ) 
        
    
            ix=ix+1
    

"""    

    
if (Fval['rcode'] == 0):
    for ix in np.arange( start=numx,stop=numx+1 ):
    
        title = f"<{valData}> {season.upper()} {years_Uv} " 
        n=ix+1
        Axes1 = Pu.axes_def(n=n,nxplo=nxplo,nyplo=nyplo, ypan=0.8 ) 
        ax1 = fig.add_axes( Axes1 )
        co1=ax1.contourf(lat_v,zlev_v,scale*Fv_sz ,levels=flev, cmap=cmap )
        co2=ax1.contour(lat_v,zlev_v, scale*Fv_sz ,levels=flev, colors='black')
        ax1.set_title( title ,fontsize=titlesize )
        ax1.set_ylim(0,plotTop)
        cb=plt.colorbar(co1)
        ax1.text(-0.08, 1.05, f"{chr(97 +n-1)})", transform=ax1.transAxes,
            fontsize=20, fontweight='bold', va='top')
"""
FigName = f"Tendencies_multiZon.png" #f"{A.exp}_{season.upper()}_{years_a[0]}-{years_a[-1]}_Dst{Dst}_zUT_climo.png"
print( FigName) 
Pu.save_link(png_filename = FigName , dpi=600 )



In [ ]:
Al[0].X